# Multivariate Gaussian Regression


## Imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch

from lightgbmlss.model import LightGBMLSS
from lightgbmlss.utils import create_mv_dataset
from lightgbmlss.distributions.Gaussian import MultivariateGaussian

## Data

We generate data with a single input and three outputs. The input and outputs have a smooth, non-monotonic relationship and correlated noise.


In [ ]:
# Generate input data
np.random.seed(42)
n_samples = 1000
x = np.linspace(0, 10, n_samples)

# Create correlated noise with increasing magnitude
def scale_cov_matrix(x, base_cov, scale_factor=0.1):
    return base_cov * (1 + scale_factor * x)


cov_matrix = np.array([[0.25, 0.15, 0.1], [0.15, 0.49, 0.2], [0.1, 0.2, 0.36]])

correlated_noise = np.array(
    [
        np.random.multivariate_normal(
            mean=[0, 0, 0], cov=scale_cov_matrix(xi, cov_matrix)
        )
        for xi in x
    ]
)

# Generate three dependent variables with smooth, non-monotonic relationships and correlated noise
y1_ = 2 * np.sin(x) + 0.5 * x
y1 = y1_ + correlated_noise[:, 0]
y2_ = 3 * np.cos(0.5 * x) + 0.3 * x**2
y2 = y2_ + correlated_noise[:, 1]
y3_ = 1.5 * np.sin(0.7 * x) * np.cos(0.3 * x) + 0.2 * x
y3 = y3_ + correlated_noise[:, 2]

# Combine the data
data = np.column_stack((x, y1, y2, y3))
real_data = np.column_stack((x, y1_, y2_, y3_))

# Plot the relationships
fig, axs = plt.subplots(3, 1, figsize=(10, 10))
fig.suptitle("Relationships between input and outputs (with correlated noise)")

for i, ax in enumerate(axs):
    ax.scatter(x, data[:, i + 1], alpha=0.5)
    ax.set_xlabel("Input")
    ax.set_ylabel(f"Output {i+1}")

plt.tight_layout()
plt.show()

print("Data shape:", data.shape)
print("First few rows of the data:")
print(data[:5])

# Plot correlation matrix of the outputs
correlation_matrix = np.corrcoef(data[:, 1:].T)
plt.figure(figsize=(8, 6))
plt.imshow(correlation_matrix, cmap="coolwarm", vmin=-1, vmax=1)
plt.colorbar()
plt.title("Correlation Matrix of Outputs")
plt.xticks(range(3), ["y1", "y2", "y3"])
plt.yticks(range(3), ["y1", "y2", "y3"])
for i in range(3):
    for j in range(3):
        plt.text(j, i, f"{correlation_matrix[i, j]:.2f}", ha="center", va="center")
plt.tight_layout()
plt.show()

### Creating special multivariate LightGBM dataset

Since LightGBM only supports a single target column, we need to create a special dataset for multivariate data.

We do this by stacking the target columns on top of each other, and repeating the feature columns for each target column. This logic is contained in the `create_mv_dataset` function.


In [ ]:
# Create lightgbm dataset
dtrain = create_mv_dataset(
    data[:, 0].reshape(-1, 1),
    data[:, 1:],
)

## Distribution Selection

We specify a Multivariate Gaussian distribution. By modifying the speciﬁcation in the following, the user can specify alternative distributional assumptions. This includes the option to choose from a wide range of parametric univariate distributions, as well as to model the data using Normalizing Flows. The user also has different function arguments for each distribution:

- `stabilization`: specifies the stabilization method for the Gradient and Hessian. Options are `None`, `MAD` and `L2`.
- `response_fn`: specifies $h_{k}(\cdot)$ and transforms the distributional parameter to the correct support. Here, we specify an exponential for $\sigma_{i}(\cdot)$ only.
- `loss_fn`: specifies the loss function used for training.  Options are `nll` (negative log-likelihood) or `crps` (continuous ranked probability score).

For additional details, see `?MultivariateGaussian`.


In [ ]:
lgblss = LightGBMLSS(
    MultivariateGaussian(stabilization="None", response_fn="exp", loss_fn="nll")
)

## Hyperparameter Optimization

Hyperparameter tuning and optimization is essential when fitting a model to all parameters of a Multivariate Gaussian at once. It's much more common for instabilities to arise with many interrelated outputs, meaning the default parameters are often not suitable.


In [ ]:
param_dict = {
    "eta":                      ["float", {"low": 1e-5,   "high": 1,     "log": True}],
    "lambda_l1":                ["float", {"low": 1e-5, "high": 100.0, "log": True}],
    "lambda_l2":                ["float", {"low": 1e-5, "high": 100.0, "log": True}],
    "min_child_samples":        ["int", {"low": 5, "high": 100, "log": False}],  # set to constant for this example
    "boosting":                 ["categorical", ["gbdt"]],
}

np.random.seed(123)
opt_param = lgblss.hyper_opt(param_dict,
                             dtrain,
                             num_boost_round=100,        # Number of boosting iterations.
                             nfold=3,                    # Number of cv-folds.
                             early_stopping_rounds=20,   # Number of early-stopping rounds
                             max_minutes=60,             # Time budget in minutes, i.e., stop study after the given number of minutes.
                             n_trials=30,               # The number of trials. If this argument is set to None, there is no limitation on the number of trials.
                             silence=True,               # Controls the verbosity of the trail, i.e., user can silence the outputs of the trail.
                             seed=123,                   # Seed used to generate cv-folds.
                             hp_seed=123                 # Seed for random number generator used in the Bayesian hyperparameter search.
                            )

In [ ]:
# Inspecting results from all trials
lgblss.study.trials_dataframe().sort_values("value")


# Model Training

We use the optimized hyper-parameters and train the model.

In [ ]:
np.random.seed(123)

opt_params = opt_param.copy()
n_rounds = opt_params["opt_rounds"]
del opt_params["opt_rounds"]

# Train Model with optimized hyperparameters
lgblss.train(opt_params,
             dtrain,
             num_boost_round=n_rounds
             )

# Prediction

Similar to a LightGBM model, we now predict from the trained model. Different options are available:

- `samples`: draws `n_samples` from the predicted distribution.
- `quantiles`: calculates quantiles from the predicted distribution.
- `parameters`: returns predicted distributional parameters.

In [ ]:
# Set seed for reproducibility
torch.manual_seed(123)

# Number of samples to draw from predicted distribution
n_samples = 1000
quant_sel = [0.05, 0.95] # Quantiles to calculate from predicted distribution

# Make predictions
preds = lgblss.predict(data[:, [0]])

# Sample from predicted distribution
pred_samples = lgblss.predict(data[:, [0]],
                              pred_type="samples",
                              n_samples=n_samples,
                              seed=123)

# Calculate quantiles from predicted distribution
pred_quantiles = lgblss.predict(data[:, [0]],
                                pred_type="quantiles",
                                n_samples=n_samples,
                                quantiles=quant_sel)

# Return predicted distributional parameters
pred_params = lgblss.predict(data[:, [0]],
                             pred_type="parameters")

In [ ]:
fig, axes = plt.subplots(3, 1, figsize=(12, 10), sharex=True)
fig.suptitle("LightGBMLSS MV Gaussian Predictions", fontsize=16)

# Compute covariance matrices
st = preds.iloc[:, 3:].to_numpy().reshape(-1, 3, 3, order="C")
cov = np.zeros_like(st)
for i in range(st.shape[0]):
    cov[i] = st[i] @ st[i].T

for i in range(3):
    ax = axes[i]
    
    mean = preds[f"loc_{i}"]
    std = np.sqrt(cov[:, i, i])
    lower = mean - 1.96 * std
    upper = mean + 1.96 * std
    
    ax.plot(data[:, [0]], mean, label="Mean")
    ax.fill_between(data[:, 0], lower, upper, alpha=0.3, label="95% CI")
    ax.scatter(data[:, 0], data[:, i+1], alpha=0.5, label="Data")
    ax.set_ylabel(f"Y{i+1}")
    ax.legend()
    ax.grid(True)

axes[-1].set_xlabel("Input")
plt.tight_layout()
plt.show()

In [ ]:
pred_samples.head()

,y_sample0,y_sample1,y_sample2,y_sample3,y_sample4,y_sample5,y_sample6,y_sample7,y_sample8,y_sample9,...,y_sample990,y_sample991,y_sample992,y_sample993,y_sample994,y_sample995,y_sample996,y_sample997,y_sample998,y_sample999
0,10.969691,8.598016,11.316233,11.196012,-0.772420,10.135987,14.128653,7.630763,12.698465,8.239548,...,10.933374,11.149927,8.531628,9.486668,10.501742,9.149441,11.645898,7.640362,9.958486,6.289453
1,9.461757,9.600316,10.147589,12.562353,16.377707,10.912953,9.325396,9.916863,7.682811,17.506680,...,10.509146,7.077166,2.610499,8.764830,16.280003,8.840451,8.440387,16.157856,9.217056,8.147191
2,9.655623,10.095434,10.497488,8.415718,10.877867,8.264465,10.300594,11.423710,10.838738,9.795403,...,11.465590,10.625634,8.727895,9.756786,8.575561,9.242844,9.997345,10.150554,10.739549,9.458581
3,7.315505,15.109694,9.312259,-3.643276,12.262859,8.238321,7.009054,6.200121,3.878844,13.135740,...,19.848660,15.314698,8.589100,12.168641,8.016486,13.352606,9.907011,8.088248,12.577316,7.965759
4,11.067125,10.200111,9.035689,8.689579,10.475836,8.348648,6.953860,7.018825,11.546731,4.925195,...,9.588341,12.277424,9.194149,13.164710,11.918127,7.771739,10.544153,9.177776,7.839355,11.506623


In [ ]:
pred_quantiles.head()

,quant_0.05,quant_0.95
0,5.405031,14.896293
1,5.210804,15.007315
2,8.298658,11.832828
3,2.962531,17.087187
4,4.807709,14.909888


In [ ]:
pred_params.head()

,loc,scale
0,9.984035,2.921586
1,9.979074,2.909918
2,9.979074,1.065636
3,9.979074,4.529788
4,9.979074,3.121158
